In [2]:
#Solve Karnaugh Maps using Q-map Solver Scraper.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from selenium import webdriver
import time

In [11]:
truth_table = ["""1	1	1	1	0	0	1
1	0	0	0	0	1	0
1	1	1	1	0	1	0
1	0	0	0	0	1	0
0	1	0	0	1	0	0
1	1	1	0	0	0	0
1	1	1	1	0	1	0
1	1	1	1	1	1	1
1	1	1	1	0	0	1
0	1	1	0	0	0	1
1	0	0	1	0	0	0
0	0	0	1	1	0	0
1	1	1	1	1	1	1
1	1	1	1	1	1	1
X	X	X	X	X	X	X
X	X	X	X	X	X	X""","""0	1	1	1	0	0	0
1	1	0	1	0	1	0
1	1	1	1	1	1	1
1	1	1	1	0	0	1
0	1	1	0	0	0	0
1	1	1	1	1	1	1
1	0	0	1	0	0	0
1	0	0	0	0	0	1
1	0	0	1	0	0	0
1	1	1	0	0	0	0
1	0	0	1	0	0	0
1	1	1	1	0	0	1
0	1	1	0	0	0	0
1	1	1	1	1	1	1
X	X	X	X	X	X	X
X	X	X	X	X	X	X""","""1	1	1	1	1	1	1
1	1	1	1	0	0	1
1	0	0	0	0	1	0
1	1	1	1	0	1	0
1	0	0	0	0	1	0
0	1	0	0	1	0	0
1	1	1	0	0	0	0
1	1	1	1	0	1	0
1	1	1	1	1	1	1
1	1	1	1	0	0	1
0	1	1	0	0	0	1
1	0	0	1	0	0	0
0	0	0	1	1	0	0
1	1	1	1	1	1	1
X	X	X	X	X	X	X
X	X	X	X	X	X	X""","""1	1	1	1	1	1	1
0	1	1	1	0	0	0
1	1	0	1	0	1	0
1	1	1	1	1	1	1
1	1	1	1	0	0	1
0	1	1	0	0	0	0
1	1	1	1	1	1	1
1	0	0	1	0	0	0
1	0	0	0	0	0	1
1	0	0	1	0	0	0
1	1	1	0	0	0	0
1	0	0	1	0	0	0
1	1	1	1	0	0	1
0	1	1	0	0	0	0
X	X	X	X	X	X	X
X	X	X	X	X	X	X"""]

LED_truth_table = ["""0	0	0	1
0	0	1	0
0	0	1	1
0	1	0	0
0	1	0	1
0	1	1	0
0	1	1	1
1	0	0	0
1	0	0	1
1	0	1	0
1	0	1	1
1	1	0	0
1	1	0	1
0	0	0	0
X	X	X	X
X	X	X	X"""]

In [12]:
def raw_one_data(truth_table):
    """Converts list() of truth table to individual set. Does for one table only."""
    truth_table = truth_table.replace("\t", ",")
    truth_table = truth_table.replace("X", "2")
    truth_table = np.array(truth_table.split("\n")) 
    new_truth_table = []
    for i in range(len(truth_table)):
        one_val = truth_table[i].split(",")
        one_row = []
        for app in one_val:
            one_row.append(int(app))
        new_truth_table.append(one_row)
    return np.array(new_truth_table)

In [13]:
def process_data(table):
    """
    Process Each truth table.
    Returns a list of processed truth tables.
    """
    processed_truth_table = []
    for one_table in table:
        processed_truth_table.append(raw_one_data(one_table))
    return np.array(processed_truth_table)

In [14]:
def data_launch(br):
    """
    launches the driver session's k-map solver webpage.
    """
    br.get("http://www.32x8.com/var4.html") 
    table = br.find_elements_by_tag_name("table")[-1]
    rows = table.find_element_by_tag_name("tbody")
    rows = rows.find_elements_by_tag_name("tr")[2:]
    return rows

In [15]:
import chromedriver_binary ##--> Change the Chrome Version Build to v.78 for executable path.
br = webdriver.Chrome(executable_path=r'C:/Users/qasim/Downloads/chromedriver_win32_v78/chromedriver.exe')
rows = data_launch(br)

In [16]:
def insert_data(elem, data):
    """
    Inserts data from the np.array() of k-maps into the table
    on chrome-driver session.
    Submits the page.
    Parameters:
    1. elem: The set of np.array() elements with input field.
    2. data: np.array() 1x16 array of data.
    """
    
    insert = elem[0:-1]
    for value, radio_web in zip(data, insert):
        row = radio_web.find_elements_by_tag_name("input")
        for i in range(len(row)):
            if(i == value):
                row[i].click()
    elem[-1].find_elements_by_tag_name("input")[-1].click()

In [17]:
def one_hex_input(elem, data):
    """
    Returns a list of K-map equations for each individual LED display.
    Parameters:
    1. elem: np.array() row object.
    2. data: 16x7 vector of k-map truth table data.
    Returns:
    1. List: np.array() object of shape => 7x1, set of equations.
    """
    num = 0
    equations = []
    dataframes = []
    new_truth_table = data.T
    for x in new_truth_table:
        insert_data(elem, x)
        sop_elem = br.find_element_by_css_selector("div#sop").find_elements_by_css_selector("div")[5].text 

        sop_elem = sop_elem.replace("A", "Y₃")
        sop_elem = sop_elem.replace("B", "Y₂")
        sop_elem = sop_elem.replace("C", "Y₁")
        sop_elem = sop_elem.replace("D", "Y₀")
        sop_elem = sop_elem.replace("y", "D[" + str(num) + "]")

        df = br.find_element_by_css_selector("div#sop").find_elements_by_css_selector("div")[2].text  
        grid = []
        df = df.split("\n")[2:]
        for spl in df:
            grid.append(spl.split(" ")[1:])
        equations.append(sop_elem)
        df = pd.DataFrame(grid, index=['00', '01', '11', '10'], columns=['00', '01', '11', '10'])
        dataframes.append(df)
        elem = data_launch(br)
        num += 1
    return np.array(equations), dataframes

In [18]:
def create_func(rows, new_truth_table):
    k_maps = []
    grids = []
    for tables in new_truth_table:
        km, gr = one_hex_input(rows, tables)
        k_maps.append(km)
        grids.append(gr)
        rows = data_launch(br)
    return np.array(k_maps), grids

In [19]:
new_truth_table = process_data(truth_table)
k_maps, grids = create_func(rows, new_truth_table)
time.sleep(1)

In [20]:
LED_rows = data_launch(br)
new_LED_truth_table = process_data(LED_truth_table)
led_kmaps, led_grids = create_func(LED_rows, new_LED_truth_table)

In [21]:
grids[0][0]

,00,01,11,10
00,1,1,1,1
01,0,1,1,1
11,1,1,x,x
10,1,0,0,1


In [22]:
k_maps[0]

array(["D[0] = Y₃'Y₂' + Y₃'Y₁ + Y₂Y₀ + Y₃Y₀'",
       "D[1] = Y₂ + Y₃'Y₀' + Y₃Y₁'",
       "D[2] = Y₃Y₁' + Y₂Y₀ + Y₂Y₁ + Y₃'Y₂'Y₀'",
       "D[3] = Y₂'Y₀' + Y₂Y₁ + Y₃Y₁ + Y₃Y₂",
       "D[4] = Y₃Y₂ + Y₂Y₁'Y₀' + Y₂Y₁Y₀ + Y₃Y₁Y₀",
       "D[5] = Y₃'Y₁ + Y₃Y₂ + Y₃'Y₂'Y₀",
       "D[6] = Y₃Y₁' + Y₂'Y₁'Y₀' + Y₂Y₁Y₀"], dtype='<U59')

In [23]:
#32 K_maps with their tables in less than 45 seconds.

In [24]:
led_grids[0][0]

,00,01,11,10
00,0,0,0,0
01,0,0,1,0
11,1,0,x,x
10,1,1,1,1


In [25]:
print("HEX Display K-Maps. Number = " + str(len(k_maps)*len(k_maps[0])))
i = 0
for maps in k_maps:
    print("HEX Display Number: " + str(i))
    for mx in maps:
        print("\n",mx)
    print("--------------------------------------")
    i += 1

HEX Display K-Maps. Number = 28
HEX Display Number: 0

 D[0] = Y₃'Y₂' + Y₃'Y₁ + Y₂Y₀ + Y₃Y₀'

 D[1] = Y₂ + Y₃'Y₀' + Y₃Y₁'

 D[2] = Y₃Y₁' + Y₂Y₀ + Y₂Y₁ + Y₃'Y₂'Y₀'

 D[3] = Y₂'Y₀' + Y₂Y₁ + Y₃Y₁ + Y₃Y₂

 D[4] = Y₃Y₂ + Y₂Y₁'Y₀' + Y₂Y₁Y₀ + Y₃Y₁Y₀

 D[5] = Y₃'Y₁ + Y₃Y₂ + Y₃'Y₂'Y₀

 D[6] = Y₃Y₁' + Y₂'Y₁'Y₀' + Y₂Y₁Y₀
--------------------------------------
HEX Display Number: 1

 D[0] = Y₀ + Y₁ + Y₃Y₂'

 D[1] = Y₃'Y₂' + Y₂'Y₀ + Y₂Y₁'

 D[2] = Y₂Y₁' + Y₃Y₀ + Y₃'Y₂'Y₀' + Y₃'Y₂'Y₁

 D[3] = Y₃'Y₂' + Y₂'Y₀' + Y₂'Y₁ + Y₁Y₀' + Y₂Y₁'Y₀

 D[4] = Y₂Y₁'Y₀ + Y₃'Y₂'Y₁Y₀'

 D[5] = Y₃'Y₁'Y₀ + Y₂Y₁'Y₀ + Y₃'Y₂'Y₁Y₀'

 D[6] = Y₁Y₀ + Y₂Y₀ + Y₃'Y₂'Y₁
--------------------------------------
HEX Display Number: 2

 D[0] = Y₂'Y₁' + Y₃'Y₀' + Y₃'Y₁ + Y₃Y₀

 D[1] = Y₂'Y₁' + Y₃'Y₀ + Y₁'Y₀ + Y₂Y₁ + Y₃Y₂'Y₀'

 D[2] = Y₂'Y₁' + Y₂Y₁ + Y₃'Y₂'Y₀ + Y₃Y₂'Y₀' + Y₃Y₁'Y₀

 D[3] = Y₂'Y₁' + Y₃Y₁' + Y₁Y₀

 D[4] = Y₃Y₂ + Y₂'Y₁'Y₀' + Y₂Y₁'Y₀

 D[5] = Y₃'Y₂'Y₀' + Y₃'Y₁'Y₀' + Y₂'Y₁'Y₀' + Y₃'Y₁Y₀ + Y₃Y₂Y₀

 D[6] = Y₂'Y₁' + Y₃Y₂'Y₀' + Y₃Y₁'

In [26]:
print("LED Display K-Maps. Number = " + str(len(led_kmaps[0])))
for lmap in led_kmaps[0]:
    print("\n",lmap)

LED Display K-Maps. Number = 4

 D[0] = Y₃Y₂' + Y₃Y₀' + Y₂Y₁Y₀

 D[1] = Y₂Y₀' + Y₂'Y₁Y₀ + Y₃'Y₂Y₁'

 D[2] = Y₁Y₀' + Y₃'Y₁'Y₀ + Y₂'Y₁'Y₀

 D[3] = Y₀'


In [27]:
grids[3][6]

,00,01,11,10
00,1,0,1,0
01,1,0,0,1
11,1,0,x,x
10,1,0,0,0


In [29]:
led_kmaps

array([["D[0] = Y₃Y₂' + Y₃Y₀' + Y₂Y₁Y₀",
        "D[1] = Y₂Y₀' + Y₂'Y₁Y₀ + Y₃'Y₂Y₁'",
        "D[2] = Y₁Y₀' + Y₃'Y₁'Y₀ + Y₂'Y₁'Y₀", "D[3] = Y₀'"]], dtype='<U34')

In [30]:
LE_E2_test = ["""0	1	1	1	0	0	0
1	1	1	1	0	0	1
1	1	0	1	0	1	0
1	0	0	0	0	1	0
1	1	1	1	1	1	1
1	1	1	1	0	1	0
0	1	1	0	0	0	0
0	1	0	0	1	0	0
1	1	1	0	0	0	0
1	0	0	1	0	0	0
1	0	0	0	0	0	1
0	1	1	0	0	0	1
0	0	0	1	1	0	0
X	X	X	X	X	X	X
X	X	X	X	X	X	X
X	X	X	X	X	X	X"""]

In [31]:
LE_rows = data_launch(br)
new_LE_E2_test = process_data(LE_E2_test)
le2_kmaps, le2_grids = create_func(LE_rows, new_LE_E2_test)

In [47]:
led_grids[0][3]

,00,01,11,10
00,1,0,0,1
01,1,0,0,1
11,1,0,x,x
10,1,0,0,1
